In [ ]:
import random

# Encoding

In [ ]:
Q = 29

In [ ]:
def encode(x):
    return x % Q

def decode(x):
    return x if x <= Q/2 else x-Q

In [ ]:
x = encode(-5)
print("encoded: %d" % x)
print("decoded: %d" % decode(x))

# Additive sharing

In [ ]:
N = 10

In [ ]:
def additive_share(secret):
    shares = [ random.randrange(Q) for _ in range(N-1) ]
    last_share = (secret - sum(shares)) % Q
    shares.append(last_share)
    return shares

def additive_reconstruct(shares):
    return sum(shares) % Q

shares = additive_share(5)
print(shares)
print(additive_reconstruct(shares))

In [ ]:
def additive_add(x, y):
    return [ (xi + yi) % Q for xi, yi in zip(x, y) ]

def additive_sub(x, y):
    return [ (xi - yi) % Q for xi, yi in zip(x, y) ]

In [ ]:
class Additive:
    
    def __init__(self, secret=None):
        self.shares = additive_share(encode(secret)) if secret is not None else []
    
    def reveal(self):
        return decode(additive_reconstruct(self.shares))
    
    def __repr__(self):
        return "Additive(%d)" % self.reveal()
    
    def __add__(x, y):
        z = Additive()
        z.shares = additive_add(x.shares, y.shares)
        return z
    
    def __sub__(x, y):
        z = Additive()
        z.shares = additive_sub(x.shares, y.shares)
        return z

x = Additive(5)
print(x)

y = Additive(8)
print(y)

z = x - y
print(z)
assert(z.reveal() == -3)

# Shamir sharing

In [ ]:
N = 10
T = 4

assert(T+1 <= N)

In [ ]:
# from http://www.ucl.ac.uk/~ucahcjm/combopt/ext_gcd_python_programs.pdf

def ext_binary_gcd(a,b):
    u, v, s, t, r = 1, 0, 0, 1, 0
    while (a % 2 == 0) and (b % 2 == 0):
        a, b, r = a//2, b//2, r+1
    alpha, beta = a, b
    while (a % 2 == 0):
        a = a//2
        if (u % 2 == 0) and (v % 2 == 0):
            u, v = u//2, v//2
        else:
            u, v = (u + beta)//2, (v - alpha)//2
    while a != b:
        if (b % 2 == 0):
            b = b//2
            if (s % 2 == 0) and (t % 2 == 0):
                s, t = s//2, t//2
            else:
                s, t = (s + beta)//2, (t - alpha)//2
        elif b < a:
            a, b, u, v, s, t = b, a, s, t, u, v
        else:
            b, s, t = b - a, s - u, t - v
    return (2 ** r) * a, s, t

def inverse(a):
    _, b, _ = ext_binary_gcd(a, Q)
    return b

In [ ]:
# using Horner's rule

def evaluate_at_point(coefs, point):
    result = 0
    for coef in reversed(coefs):
        result = coef + point * result
        result %= Q
    return result

In [ ]:
# see https://en.wikipedia.org/wiki/Lagrange_polynomial

def interpolate_at_zero(points_values):
    points, values = zip(*points_values)
    result = 0
    for i in range(len(values)):
        xi = points[i]
        yi = values[i]
        num = 1
        denum = 1
        for j in range(len(values)):
            if j != i:
                xj = points[j]
                num = (num * xj) % Q
                denum = (denum * (xj - xi)) % Q
        result = (result + yi * num * inverse(denum)) % Q
    return result

In [ ]:
def sample_shamir_polynomial(zero_value):
    coefs = [zero_value]
    coefs.extend( random.randrange(Q) for _ in range(T) )
    return coefs

In [ ]:
def shamir_share(secret):
    coefs = sample_shamir_polynomial(secret)
    return [ evaluate_at_point(coefs, point) for point in range(1, N+1) ]

def shamir_reconstruct(shares):
    points = range(1, N+1)
    values = shares
    points_values = [ (point, value) for point, value in zip(points, values) if value is not None ]
    return interpolate_at_zero(points_values)

s = shamir_share(5)
s[-1] = None
s[-2] = None
s[-3] = None
s[-4] = None
x = shamir_reconstruct(s)
assert(x == 5)

In [ ]:
def shamir_add(x, y):
    return [ (xi + yi) % Q for xi, yi in zip(x, y) ]

def shamir_sub(x, y):
    return [ (xi - yi) % Q for xi, yi in zip(x, y) ]

In [ ]:
def shamir_mul(x, y):
    return [ (xi * yi) % Q for xi, yi in zip(x, y) ]

In [ ]:
class Shamir:
    
    def __init__(self, secret=None):
        self.shares = shamir_share(encode(secret)) if secret is not None else []
        self.degree = T
    
    def reveal(self):
        assert(self.degree+1 <= N)
        return decode(shamir_reconstruct(self.shares))
    
    def __repr__(self):
        return "Shamir(%d)" % self.reveal()
    
    def __add__(x, y):
        z = Shamir()
        z.shares = shamir_add(x.shares, y.shares)
        z.degree = max(x.degree, y.degree)
        return z
    
    def __sub__(x, y):
        z = Shamir()
        z.shares = shamir_sub(x.shares, y.shares)
        z.degree = max(x.degree, y.degree)
        return z
    
    def __mul__(x, y):
        z = Shamir()
        z.shares = shamir_mul(x.shares, y.shares)
        z.degree = x.degree + y.degree
        return z
    
x = Shamir(2)
print(x)

y = Shamir(3)
print(y)

z = x - y
print(z)
assert(z.reveal() == -1)

v = x * y
print(v)
assert(v.reveal() == 6)

# Packed sharing

In [ ]:
N = 10
T = 5
K = 3

assert(T+K <= N)

In [ ]:
def interpolation_at_point(points_values, point):
    points, values = zip(*points_values)
    result = 0
    for i in range(len(values)):
        xi = points[i]
        yi = values[i]
        num = 1
        denum = 1
        for j in range(len(values)):
            if j != i:
                xj = points[j]
                num = (num * xj) % Q
                denum = (denum * (xj - xi)) % Q
        result = (result + yi * num * inverse(denum)) % Q
    return result

In [ ]:
def sample_packed_polynomial(secrets):
    assert(len(secrets) == K)
    points = secrets
    points.extend( random.randrange(Q) for _ in range(T) )